# Bias Model for FEVER

In [1]:
import json
from random import random
import os
from typing import Dict, List

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import sys
sys.path.append("../")
from my_package.models.traditional.classifier import Classifier
from my_package.utils.handcrafted_features.counter import count_negations
from my_package.utils.handcrafted_features.overlap import get_lexical_overlap, get_entities_overlap

/home/patomp/anaconda3/envs/envFakeNews/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Configurations

In [2]:
DUMMY_PREFIX = "" # "sample_" for example and "" for the real one

TRAIN_DATA_FILE = "../data/paraphrase_identification/%sqqp.train.jsonl"%DUMMY_PREFIX
DEV_DATA_FILE = "../data/paraphrase_identification/%sqqp.dev.jsonl"%DUMMY_PREFIX
TEST_DATA_FILE = "../data/paraphrase_identification/paws.dev_and_test.jsonl"

WEIGHT_KEY = "sample_weight"
OUTPUT_TRAIN_DATA_FILE = "../data/paraphrase_identification/%sweighted_qqp.train.jsonl"%DUMMY_PREFIX
SAVED_MODEL_PATH = "../results/qqp/bias_model"

DOC1_KEY = "sentence1"
DOC2_KEY = "sentence2"
LABEL_KEY = "is_duplicate"

POSSIBLE_LABELS = ("0", "1")
BIAS_CLASS = "1"

MAX_SAMPLE = -1 # -1 for non-maximal mode or a finite number e.g. 2000
DROP_RATE = 0.0
TEST_FRAC = 0.2

MAX_TEST_SAMPLE = -1

In [3]:
def inference_prob_to_index(x: List[Dict[str, float]]) -> List[float]:
    return [
        x["0"],
        x["1"]
    ]

## Dataset

In [4]:
def read_data(
    file: str = TRAIN_DATA_FILE,
    sent1_key: str = DOC1_KEY,
    sent2_key: str = DOC2_KEY,
    label_key: str = LABEL_KEY,
    drop_rate: float = 0.0
):
    docs = []
    labels = []

    N_SAMPLE = 0

    with open(file, 'r') as fh:
        line = fh.readline()
        while line:
            if random() > drop_rate:
                datapoint = json.loads(line)
                docs.append([datapoint[sent1_key], datapoint[sent2_key]])
                labels.append(str(datapoint[label_key]))

                N_SAMPLE += 1
                if MAX_SAMPLE != -1 and N_SAMPLE == MAX_SAMPLE:
                    break
            line = fh.readline()
    print("# samples: ", N_SAMPLE)
    return docs, labels

In [5]:
docs, labels = read_data(drop_rate=DROP_RATE)

# samples:  2000


In [6]:
docs[:2]

[['How can I stop my dog from chewing my shoes?',
  'How do you stop a English Bulldog/Pitbull mix puppy from biting my shoes?'],
 ['What are the most interesting products and innovations that Guess is coming out with in 2016?',
  'What are the most interesting products and innovations that Twitter is coming out with in 2016?']]

In [7]:
labels[:2]

['1', '0']

In [8]:
docs_train, docs_test, labels_train, labels_test = train_test_split(
    docs, labels,
    stratify=labels, test_size=TEST_FRAC,
    random_state=42
)

## Bias Classifier

In [9]:
feature_extractors = [
    get_lexical_overlap,
    get_entities_overlap
]

In [10]:
config = {
    "n_grams": [1, 2],
    "top_ks": [50, 50], # select by LMI
    "verbose": True
}

In [11]:
classifier = Classifier(
    possible_labels=POSSIBLE_LABELS,
    feature_extractors=feature_extractors,
    config=config
)

## Training

In [12]:
classifier.fit(docs_train, labels_train)
classifier.save(SAVED_MODEL_PATH)

------ Top N-grams for sentence 1 ------
1-gram LMI:  {'1': {'how': 0.0031326028947570458, 'do': 0.0022861665765403195, 'can': 0.0018635978552370462, 'you': 0.0010961121901843784, 'i': 0.001094121746698343, '500': 0.0010926629662799143, 'quora': 0.0008672703477951083, 'notes': 0.0008355657977434639, 'best': 0.000781103511897613, 'life': 0.0007218353413456668, '1000': 0.0007172359880179036, 'money': 0.0007016123840321265, 'your': 0.0006490025244530314, 'improve': 0.000640805035180602, 'war': 0.00063930085172901, 'who': 0.0006348082060698078, 'think': 0.0006036014370502948, '2000': 0.0005784686292070134, 'rupee': 0.0005784686292070134, 'rs': 0.000557724477531417, 'ever': 0.0005437259735849995, 'really': 0.0005251035191321394, 'english': 0.0005150140500252845, 'is': 0.00048535574681406196, 'battle': 0.00044992004493878823, 'my': 0.00041669792055320105, 'ways': 0.0004137301347816252, 'india': 0.0004038868569796261, 'president': 0.0003922159970314348, 'currency': 0.00038564575280467564, 'ge

2-gram LMI:  {'1': {'how_do': 0.0016769479957734275, 'how_can': 0.0016401141655068615, 'do_i': 0.0014621707038709555, 'what_is': 0.0008920248299538186, 'can_i': 0.0008532672839026691, 'what_are': 0.0007200096490867508, 'the_best': 0.0007022899896785075, 'on_quora': 0.0006349114134642144, 'how_did': 0.0005442978836141374, 'is_the': 0.0005244353137581608, 'will_the': 0.0005226440803104914, '500_and': 0.0005081760779579652, 'and_1000': 0.0005081760779579652, 'ways_to': 0.00047583312478553504, 'donald_trump': 0.00047583312478553504, 'i_get': 0.00044957676256034487, 'are_the': 0.0004392461393483449, 'are_some': 0.00043922357199322376, '1000_notes': 0.00043557949539254167, 'the_ban': 0.00043557949539254167, 'can_we': 0.0004236085736049264, 'make_money': 0.0003996504149125401, 'do_you': 0.0003977486430961743, 'the_indian': 0.0003785218717673998, 'can_one': 0.00036298291282711807, 'ban_of': 0.00036298291282711807, 'rupee_notes': 0.00036298291282711807, 'some_good': 0.000354152351395897, 'impro

In [13]:
# Inferential Examples
x = [['Roman Atwood is a content creator .',
  'He is best known for his vlogs , where he posts updates about his life on a daily basis .'],
 ['Roman Atwood is a content creator .',
  "He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks ."]] 
classifier.inference(x)

[{'0': 0.9326622042463655, '1': 0.06733779575363455},
 {'0': 0.9184946160216139, '1': 0.0815053839783861}]

## Evaluation

In [14]:
y_preds = classifier.predict(docs_test)

print("Accuracy on original train set: %.3f"% accuracy_score(labels_test, y_preds))
print("F1-pos on original train set: %.3f"% f1_score(labels_test, y_preds, pos_label='1'))
print("F1-neg on original train set: %.3f"% f1_score(labels_test, y_preds, pos_label='0'))

Accuracy on original train set: 0.682
F1-pos on original train set: 0.506
F1-neg on original train set: 0.766


In [15]:
# validate load process of the classifier
test_classifier = Classifier(
    possible_labels=POSSIBLE_LABELS,
    feature_extractors=feature_extractors,
    config=config
)
test_classifier.load(SAVED_MODEL_PATH)

y_preds = classifier.predict(docs_test)

print("Accuracy on original train set: %.3f"% accuracy_score(labels_test, y_preds))
print("F1-pos on original train set: %.3f"% f1_score(labels_test, y_preds, pos_label='1'))
print("F1-neg on original train set: %.3f"% f1_score(labels_test, y_preds, pos_label='0'))

Accuracy on original train set: 0.682
F1-pos on original train set: 0.506
F1-neg on original train set: 0.766


### On original test set

In [16]:
eval_docs, eval_labels = read_data(DEV_DATA_FILE)

# samples:  200


In [17]:
y_preds = classifier.predict(eval_docs)

print("Accuracy on original test set: %.3f"% accuracy_score(eval_labels, y_preds))
print("F1-pos on original test set: %.3f"% f1_score(eval_labels, y_preds, pos_label='1'))
print("F1-neg on original test set: %.3f"% f1_score(eval_labels, y_preds, pos_label='0'))

Accuracy on original test set: 0.705
F1-pos on original test set: 0.550
F1-neg on original test set: 0.781


### On symmetric (challenge) test set

In [18]:
eval_docs, eval_labels = read_data(TEST_DATA_FILE)

# samples:  677


In [19]:
y_preds = classifier.predict(eval_docs)

print("Accuracy on challenge test set: %.3f"% accuracy_score(eval_labels, y_preds))
print("F1-pos on challenge test set: %.3f"% f1_score(eval_labels, y_preds, pos_label='1'))
print("F1-neg on challenge test set: %.3f"% f1_score(eval_labels, y_preds, pos_label='0'))

Accuracy on challenge test set: 0.409
F1-pos on challenge test set: 0.419
F1-neg on challenge test set: 0.399


## Write predicted probability to the training set

In [20]:
def get_weight(prob_score_bias_class: float, ground_truth_label: str, bias_label: str = BIAS_CLASS) -> float:
    assert type(ground_truth_label) == type(bias_label)
    if ground_truth_label == bias_label:
        return 1/prob_score_bias_class
    return 1/(1-prob_score_bias_class)

In [21]:
# test cases
def test_bias_label():
    weight = get_weight(0.2, "A", bias_label="A")
    np.testing.assert_almost_equal(1/0.2, weight, 5)
    
def test_not_bias_label():
    weight = get_weight(0.2, "B", bias_label="A")
    np.testing.assert_almost_equal(1/0.8, weight, 5)
    
test_bias_label()
test_not_bias_label()

In [22]:
f_output = open(OUTPUT_TRAIN_DATA_FILE, 'w')

In [23]:
N_SAMPLE = 0

with open(TRAIN_DATA_FILE, 'r') as fh:
    line = fh.readline()
    while line:
        datapoint = json.loads(line)
        x = [[datapoint[DOC1_KEY], datapoint[DOC2_KEY]]]
        
        probs = classifier.inference(x)[0]
        prob = probs[BIAS_CLASS]
        weight = get_weight(
            prob_score_bias_class=prob,
            ground_truth_label=str(datapoint[LABEL_KEY]),
            bias_label=BIAS_CLASS
        )
        if datapoint.get("weight", None) != None:
            del datapoint["weight"] # only for fever
        f_output.write("%s\n"%json.dumps({
            **datapoint,
            WEIGHT_KEY: weight,
            "bias_probs": inference_prob_to_index(probs),
            "bias_prob": prob
        }))

        N_SAMPLE += 1
        if MAX_SAMPLE != -1 and N_SAMPLE == MAX_SAMPLE:
            break
        line = fh.readline()
        
f_output.close()